In [1]:
%matplotlib notebook

import torch
import torch.nn as nn
import torch.nn.functional as F
import math as ms
import numpy as np
from numpy import random
from d2l import torch as d2l
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import os
from matplotlib.animation import FuncAnimation
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# torch.manual_seed(0)
# np.random.seed(0)

# This concept is also called teacher forceing.
# The flag decides if the loss will be calculted over all
# or just the predicted values.
calculate_loss_over_all_values = False
cypl = 200
batch_size = 80 # batch size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""1.同时产生多D数据污染
2.对D需要进行编码
"""

def get_data():
    data = pd.read_csv('E:/论文/数据/fx3/wz50_pos_noise_35.0000.csv', encoding='GBK', chunksize=100,header = None) 
    data = pd.concat(data, ignore_index=True)
    data = data.loc[:, :]

    train_data, test_data = train_test_split(data, train_size=0.8, random_state=np.random.randint(1,100))

    train_data =torch.tensor(np.array(train_data)).float()
#     train_label =train_data[:,-1].reshape(-1,1)
#     train_seq = train_data[:,0:-1] 

    test_data =torch.tensor(np.array(test_data)).float()
#     test_label =test_data[:,-1].reshape(-1,1)
#     test_seq = test_data[:,0:-1] 


#     train_sequence = torch.cat((train_seq,train_label),dim =1).float()
#     test_sequence = torch.cat((test_seq,test_label),dim =1).float()
#     train_sequence = train_sequence
#     test_sequence = test_sequence
    return train_data, test_data
def input_trans(data,batch_size):
    data = data.reshape(cypl,batch_size)
#     data = np.array(data.cpu()).T
#     data = torch.tensor(np.diff(data)).t()
    data=data.cpu()
#     data = torch.cat((data,torch.zeros([1,len(data[0])])),dim = 0)
    
#     data = torch.clamp(data,min=0.0)
    data = data.reshape(cypl,batch_size,1).to(device)
    return data

def noramlization(data):
    
    return (data - torch.mean(data))/torch.std(data)

def get_batch(source,i, batch_size):
  #  seq_len = min(batch_size, len(source) - 1 - i)
    if i + batch_size > len(source):  # 如果不够，直接返回None
        return None, None
    else:
        seq_len = batch_size
        data = source[i:i + seq_len]
        target = data[:,-1]
        data1 = data[:,0:cypl]
        data1 = data1.t().to(device)
        target = target.reshape(1,batch_size).to(device)
        return data1,target
def time_embedding_pos(data,pos):
    data = data.reshape(data.shape[0],data.shape[1])
    data =data.t()
    pos = pos.transpose(0,1)
    sfr = data
    box = torch.zeros(data.shape[0],data.shape[1],cypl)
    box[:,:,0] = sfr
    for j in range(len(box[0,:,0])):
        box[:,j,1:j+1] = torch.flip(box[:,0:j,0], dims=[1])
    box = box.to(device)
    posi = pos[:,:,1:]
    
    box[:,:,1:] += posi
    box = box.transpose(0,1).to(device)
    return box
def dis(src):
    d = src.shape[0]
    for i in range(d):
        src[i,:,1:] = src[i,:,1:] * src[i,-1,0]
    return src
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=cypl):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-ms.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        # pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
     
        fd = x + self.pe[:x.size(0), :]
        
        return fd

class TransAm(nn.Module):
    def __init__(self, feature_size=12, num_layers=6, dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(cypl)
        self.lru = nn.LeakyReLU(negative_slope=0.01, inplace=False)
        
        self.Relu=nn.ReLU()
        self.maxpooling = torch.nn.MaxPool1d(6, stride=1, padding=1)
        self.BN = nn.BatchNorm1d(300)
        self.cov1 = nn.Conv1d(in_channels=cypl,out_channels=200,kernel_size=10,padding = 8)
        self.cov2 = nn.Conv1d(in_channels=200,out_channels=100,kernel_size=3,padding = 2)
        self.cov3 = nn.Conv1d(in_channels=100,out_channels=12,kernel_size=1,padding = 0)
        self.cov_block = nn.Sequential(self.cov1,self.Relu,self.maxpooling,
                                       self.cov2,self.Relu,self.maxpooling,
                                       self.cov3,self.Relu,self.maxpooling)
#         self.cov4 = nn.Conv1d(in_channels=300,out_channels=200,kernel_size=3,padding = 1)
#         self.cov5 = nn.Conv1d(in_channels=200,out_channels=100,kernel_size=3,padding = 1)
#         self.cov6 = nn.Conv1d(in_channels=100,out_channels=10,kernel_size=3,padding = 1)

#         self.cov_block1 = nn.Sequential(self.cov4,self.Relu,self.maxpooling,
#                                        self.cov5,self.Relu,self.maxpooling,
#                                        self.cov6,self.Relu,self.maxpooling)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=6, dropout=0,activation = "gelu",
                                                       batch_first = False,norm_first = False,dim_feedforward = 200)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder = nn.Sequential(nn.Linear(12, 5),nn.Linear(5, 1))
        self.decoder2 = nn.Sequential(nn.Linear(cypl,120),nn.Tanh(),
                                      nn.Linear(120,100),nn.Tanh(),
                                      nn.Linear(100,10),nn.Tanh(),
                                      nn.Linear(10,2),nn.Tanh()
                                     )

        self.init_weights()


    def init_weights(self):
        initrange = 0.4
        
        self.decoder[0].bias.data.zero_()
        self.decoder[0].weight.data.uniform_(-initrange, initrange)
        for i in range(0,len(self.decoder2),2):
            self.decoder2[i].bias.data.zero_()
            self.decoder2[i].weight.data.uniform_(-initrange, initrange)
    def forward(self, src):


        output = self.pos_encoder(src)
#         output = dis(output)
#         src = output.transpose(0,2).transpose(0,1)

        output = self.cov_block(output.transpose(0,2).transpose(0,1))

#         output = output +src
#         output size = (batch_size,emb,time len )
#         output = self.cov_block1(output)

        output = self.transformer_encoder(output.transpose(0,2).transpose(1,2)) 

#         output = output.transpose(0,2).transpose(1,2)

        output = self.decoder(output.transpose(0,1)).squeeze(2)
#         output = noramlization(output)


        
#         output = output.squeeze(2)
    
        output = self.decoder2[0](output.float())

        for i in range(1,len(self.decoder2)):
            
            output = self.decoder2[i](output)
            output = noramlization(output)
            

        
        return output

def train(train_data,lost,acr,model,lossfix,lr,optimizer,scheduler,epochs,epoch):
    model.train()
    total_loss = 0.
    start_time = time.time()
    
    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        
        data, targets = get_batch(train_data, i, batch_size)
        optimizer.zero_grad()
        data = input_trans(data,batch_size) 
        data = noramlization(data)
        output = model(data).float()
        targets = targets.reshape([batch_size]).long()
        loss = lossfix(output,targets).sum()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        lost.append(float(loss))
        acr.append(accuracy(output,targets))
        log_interval = int(len(train_data) / batch_size / 4)
        total_loss += loss.item()
#             cur_loss = total_loss / log_interval
        elapsed = time.time() - start_time
        if batch % log_interval == 0 and batch > 0:
            print('| epoch {:3d} |  '
                  'lr {:02.6f} |  '

                  'loss {:5.5f}| acc {:2f}'.format(
                epoch,   scheduler.get_lr()[0],

                loss,accuracy(output,targets)))  # , math.exp(cur_loss)
            total_loss = 0
            start_time = time.time()
            
def accuracy(y_hat, y):
    m = nn.Softmax(dim = 1)
    y_hat = m(y_hat)
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return round(float(cmp.type(y.dtype).sum())/len(y.t()),6)
   # pre_output = y_hat[torch.tensor(range(0,len(y_hat))).type(torch.int64),y.type(torch.int64)]
   #  return pre_output

def train_informer():
    lossfix=nn.CrossEntropyLoss()
    lossfix = lossfix.to(device)
    model = TransAm().to(device)
    lr = 1*batch_size/256
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 3, gamma=0.75)

    best_val_loss = float("inf")
    epochs = 100 # The number of epochs
    best_model = None
    lost = []
    acr = []
    ts_lost = []
    train_data, test_data = get_data()
    for epoch in range(1, epochs + 1):


        train_data=train_data[torch.randperm(train_data.size(0))]


        print('-' * 89)

        print('-' * 89)

        epoch_start_time = time.time()
        train(train_data,lost,acr,model,lossfix,lr,optimizer,scheduler,epochs,epoch)

        test_acc = 0

        for i in range(20):

            tst,tar = get_batch(test_data,np.random.randint(1,len(test_data) - batch_size - 1),batch_size)

            tst = input_trans(tst,batch_size)
            tst = noramlization(tst)
            tst_output = model(tst).float()
            test_acc += accuracy(tst_output,tar)
        ts_lost.append(test_acc/20) 
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test_acc: {:5.6f} |  '.format(
            epoch, (time.time() - epoch_start_time),test_acc/20
           ))  # , math.exp(val_loss) | valid ppl {:8.2f}
        print('-' * 89)

        if test_acc/20 >=0.99 :
            torch.save(model, 'T_99.pt')
        if test_acc/20 >= 0.95 and test_acc/20<0.99:
            torch.save(model, 'T_95.pt')
        if test_acc/20 >=0.85 and test_acc/20<0.95:
            torch.save(model, 'T_85.pt')
    
if __name__ == '__main__':
    train_informer()

 







-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------


D:\Users\17306\miniconda3\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |  lr 0.312500 |  loss 0.12912| acc 1.000000
| epoch   1 |  lr 0.312500 |  loss 0.12909| acc 1.000000
| epoch   1 |  lr 0.312500 |  loss 0.13076| acc 1.000000
| epoch   1 |  lr 0.312500 |  loss 0.12788| acc 1.000000
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 11.54s | test_acc: 0.998125 |  
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| epoch   2 |  lr 0.312500 |  loss 0.14937| acc 0.987500
| epoch   2 |  lr 0.312500 |  loss 0.14901| acc 0.987500


KeyboardInterrupt: 